In [ ]:
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import BatchNormalization,Conv2DTranspose,Dropout
from keras.utils.vis_utils import plot_model
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip,ElasticTransform,GridDistortion,RandomRotate90,Transpose,OpticalDistortion,RandomSizedCrop,CenterCrop,Crop,PadIfNeeded

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import cv2
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input,Conv2D,Conv2DTranspose,Dropout,Concatenate,LeakyReLU,BatchNormalization,Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.metrics import Recall,Precision

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
from sklearn.utils import shuffle
def shuffling(x,y):
    x,y = shuffle(x,y)
    return x,y

In [ ]:
from sklearn.model_selection import train_test_split
def load_path(path,split=0.2):
    X = sorted(glob(os.path.join(path,"images","*.jpg")))
    Y = sorted(glob(os.path.join(path,"masks","*.jpg")))
    
    size = int(len(X) * split)
    train_x,valid_x = train_test_split(X,test_size=size,random_state=1)
    train_y,valid_y = train_test_split(Y,test_size=size,random_state=1)

    train_x,test_x = train_test_split(train_x,test_size=size,random_state=1)
    train_y,test_y = train_test_split(train_y,test_size=size,random_state=1)

    return (train_x,train_y), (valid_x,valid_y), (test_x,test_y)



In [ ]:
path = '/content/drive/MyDrive/DSB'
(train_x,train_y), (valid_x,valid_y), (test_x,test_y) = load_path(path)

In [ ]:
print(len(train_x),'-',len(train_y))
print(len(valid_x),'-',len(valid_y))
print(len(test_x),'-',len(test_y))

402 - 402
134 - 134
134 - 134


In [ ]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    x = cv2.resize(x,(256,256))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [ ]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(256,256))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x,axis=-1) #(256,256,1)
    return x

In [ ]:
def tf_parse(x,y):
    def _parse(x,y):
        x= read_image(x)
        y = read_mask(y)
        return x,y
    x,y = tf.numpy_function(_parse,[x,y],[tf.float32,tf.float32])
    x.set_shape([256,256,3])
    y.set_shape([256,256,1])
    return x,y
    

In [ ]:
def tf_dataset(X,Y,batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
# directory to save files
create_dir("/content/drive/MyDrive/cellNuclei/files")

#Hyperparameters
batch_size=2
lr = 1e-4
num_epochs = 100
model_path = os.path.join("/content/drive/MyDrive/cellNuclei/files","model.h5")
csv_path = os.path.join("/content/drive/MyDrive/cellNuclei/files","data.csv")

In [ ]:
train_dataset = tf_dataset(train_x,train_y)
valid_dataset = tf_dataset(valid_x,valid_y)

In [ ]:
#no of steps
train_steps = len(train_x)//batch_size
test_steps = len(valid_x)//batch_size

In [ ]:
print(train_steps)
print(test_steps)

201
67


In [ ]:
if len(train_x) % batch_size != 0:
    train_steps +=1
if len(valid_x) % batch_size != 0:
    test_steps +=1

In [ ]:
print(train_steps)
print(test_steps)

201
67


In [ ]:
def iou(y_true,y_pred):
    def f(y_true,y_pred):
        intersection = (y_true*y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f,[y_true,y_pred],tf.float32)

In [ ]:
smooth = 1e-15
def dice_coef(y_true,y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true*y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

In [ ]:
def dice_loss(y_true,y_pred):
    return 1.0 - dice_coef(y_true,y_pred)

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-aznp4k9d
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-aznp4k9d
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=d122bfa80b3e0a619840b37e9d24d0d9b5e7503d91fe7375a65be0ee2071bea6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ultvpfwy/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

In [ ]:

def conv_block(inputs,num_filters):
  init = RandomNormal(stddev=0.02)
  x = Conv2D(num_filters,3,padding='same',kernel_initializer=init)(inputs)
  x = InstanceNormalization(axis=-1)(x)
  x = Activation('relu')(x)

  x = Conv2D(num_filters,3,padding='same',kernel_initializer=init)(x)
  x = InstanceNormalization(axis=-1)(x)
  x = Activation('relu')(x)

  return x

In [ ]:
def define_decoder(inputs,skip_layer,num_filters):
  init = RandomNormal(stddev=0.02)
  x = Conv2DTranspose(num_filters,(2,2),strides=(2,2),padding='same',kernel_initializer=init)(inputs)
  g = Concatenate()([x,skip_layer])
  g = conv_block(g,num_filters)
  return g

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input,Conv2DTranspose,BatchNormalization,Dropout,Concatenate,Activation,Conv2D
from keras.models import Model
from keras.initializers import RandomNormal

In [ ]:
def vgg16_unet(input_shape):
  inputs = Input(shape=input_shape)
  vgg16 = VGG16(include_top=False,weights='imagenet',input_tensor=inputs)

  # We will extract encoder layers based on their output shape from vgg16 model
  s1 = vgg16.get_layer('block1_conv2').output  #512
  s2 = vgg16.get_layer('block2_conv2').output  #256
  s3 = vgg16.get_layer('block3_conv3').output  #128
  s4 = vgg16.get_layer('block4_conv3').output  #64

  # bottleneck/bridege layer from vgg16
  b1 = vgg16.get_layer('block5_conv3').output #32

  # Decoder Block 
  d1 = define_decoder(b1,s4,512)
  d2 = define_decoder(d1,s3,256)
  d3 = define_decoder(d2,s2,128)
  d4 = define_decoder(d3,s1,64)

  #output layer 
  outputs = Conv2D(1,1,padding='same',activation='sigmoid')(d4)

  model = Model(inputs,outputs)

  return model

In [ ]:
model = vgg16_unet((256,256,3))
model.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
_____________________________

In [ ]:
model.compile(loss=dice_loss,optimizer=Adam(lr),metrics=[dice_coef,iou,Recall(),Precision()])

In [ ]:
model.fit(
        train_dataset,
        epochs=150,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=test_steps)

Epoch 1/150
201/201 [==============================] - 436s 2s/step - loss: 0.4797 - dice_coef: 0.5203 - iou: 0.3769 - recall: 0.7151 - precision: 0.6549 - val_loss: 0.2923 - val_dice_coef: 0.7077 - val_iou: 0.5712 - val_recall: 0.7317 - val_precision: 0.8030
Epoch 2/150
201/201 [==============================] - 117s 583ms/step - loss: 0.2673 - dice_coef: 0.7327 - iou: 0.5955 - recall: 0.7264 - precision: 0.8233 - val_loss: 0.2178 - val_dice_coef: 0.7822 - val_iou: 0.6586 - val_recall: 0.7159 - val_precision: 0.8540
Epoch 3/150
201/201 [==============================] - 117s 581ms/step - loss: 0.2088 - dice_coef: 0.7912 - iou: 0.6675 - recall: 0.7138 - precision: 0.8631 - val_loss: 0.1807 - val_dice_coef: 0.8193 - val_iou: 0.7059 - val_recall: 0.7091 - val_precision: 0.8776
Epoch 4/150
201/201 [==============================] - 117s 580ms/step - loss: 0.1755 - dice_coef: 0.8245 - iou: 0.7114 - recall: 0.7079 - precision: 0.8826 - val_loss: 0.1628 - val_dice_coef: 0.8372 - val_iou: 0.7

KeyboardInterrupt: ignored

In [ ]:
model.fit(
        train_dataset,
        epochs=20,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=test_steps)

Epoch 1/20
201/201 [==============================] - 117s 581ms/step - loss: 0.1178 - dice_coef: 0.8822 - iou: 0.7950 - recall: 0.6991 - precision: 0.9116 - val_loss: 0.1174 - val_dice_coef: 0.8826 - val_iou: 0.7945 - val_recall: 0.6986 - val_precision: 0.9141
Epoch 2/20
201/201 [==============================] - 117s 581ms/step - loss: 0.1110 - dice_coef: 0.8890 - iou: 0.8052 - recall: 0.6980 - precision: 0.9161 - val_loss: 0.1149 - val_dice_coef: 0.8851 - val_iou: 0.7984 - val_recall: 0.6978 - val_precision: 0.9180
Epoch 3/20
201/201 [==============================] - 117s 580ms/step - loss: 0.1049 - dice_coef: 0.8951 - iou: 0.8148 - recall: 0.6974 - precision: 0.9195 - val_loss: 0.1037 - val_dice_coef: 0.8963 - val_iou: 0.8157 - val_recall: 0.6970 - val_precision: 0.9213
Epoch 4/20
201/201 [==============================] - 116s 580ms/step - loss: 0.1000 - dice_coef: 0.9000 - iou: 0.8226 - recall: 0.6964 - precision: 0.9228 - val_loss: 0.0986 - val_dice_coef: 0.9014 - val_iou: 0.82

In [ ]:
model.fit(
        train_dataset,
        epochs=25,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=test_steps)

Epoch 1/25
201/201 [==============================] - 116s 577ms/step - loss: 0.0774 - dice_coef: 0.9226 - iou: 0.8597 - recall: 0.6929 - precision: 0.9435 - val_loss: 0.0859 - val_dice_coef: 0.9141 - val_iou: 0.8445 - val_recall: 0.6929 - val_precision: 0.9438
Epoch 2/25
201/201 [==============================] - 116s 577ms/step - loss: 0.0742 - dice_coef: 0.9258 - iou: 0.8652 - recall: 0.6929 - precision: 0.9440 - val_loss: 0.0873 - val_dice_coef: 0.9127 - val_iou: 0.8423 - val_recall: 0.6930 - val_precision: 0.9443
Epoch 3/25
201/201 [==============================] - 116s 578ms/step - loss: 0.0737 - dice_coef: 0.9263 - iou: 0.8661 - recall: 0.6930 - precision: 0.9445 - val_loss: 0.0849 - val_dice_coef: 0.9151 - val_iou: 0.8463 - val_recall: 0.6931 - val_precision: 0.9448
Epoch 4/25
201/201 [==============================] - 116s 579ms/step - loss: 0.0726 - dice_coef: 0.9274 - iou: 0.8680 - recall: 0.6931 - precision: 0.9450 - val_loss: 0.0870 - val_dice_coef: 0.9130 - val_iou: 0.84

In [ ]:
model.save('vggUnetNucleiSeg50Epochs.h5')